In [1]:
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
#---------------------------------------------------------#
# Prototyping
#---------------------------------------------------------#

In [3]:
eds_cols = {}
window = 7

In [9]:
# file location for the csv file
file = "./testing_data.csv"
# return pandas dataframe of the csv file
df = pd.read_csv(file)
# remove all NaN entries
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
a=df.iterrows()
i = 0
for index,row in a:
    if math.isnan(row[2]):
        break
    else:
        i = i + 1
table = df[0:i]
df.head(10)

,Date,Time,Temperature(C),Humidity(%),GPOA(W/M2),EDS(#),OCV_Before(V),OCV_After(V),SCC_Before(A),SCC_After(A),CTRL1_OCV(V),CTRL1_SCC(A),CTRL2_OCV(V),CTRL2_SCC(A),EDS_PWR_Before(W),EDS_PWR_After(W),CTRL1_PWR(W),CTRL2_PWR(W)
0,1/22/2020,12:33:26,32.4,28.3,-1,1,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
1,1/22/2020,12:34:51,32.4,28.0,-1,2,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
2,1/22/2020,12:37:11,31.3,27.4,-1,3,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
3,1/22/2020,12:38:35,31.3,45.5,-1,4,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
4,1/22/2020,12:40:58,31.0,29.9,-1,5,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925
5,1/23/2020,12:33:26,32.4,28.3,-1,1,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
6,1/23/2020,12:34:51,32.4,28.0,-1,2,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
7,1/23/2020,12:37:11,31.3,27.4,-1,3,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
8,1/23/2020,12:38:35,31.3,45.5,-1,4,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
9,1/23/2020,12:40:58,31.0,29.9,-1,5,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925
